In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numba import njit

In [2]:
@njit
def acf(x, length=20):
    return np.array([1]+[np.corrcoef(x[:-i], x[i:])[0,1] for i in range(1, length)])

@njit
def CF(d, n):
    if np.sqrt(d) != np.floor(np.sqrt(d)):

        a = np.zeros(n)
        r = np.zeros(n)
        u = np.zeros(n)
        v = np.zeros(n)

        a[0] = int(np.floor(np.sqrt(d)))
        u[0] = a[0]
        v[0] = 1
        r[0] = (np.sqrt(d) - u[0])/v[0]

        for i in np.arange(1,n):
            a[i] = int(np.floor(1/r[i-1]))
            v[i] = int((d - u[i-1]*u[i-1])/v[i-1])
            u[i] = a[i]*v[i] - u[i-1]
            r[i] = (np.sqrt(d) - u[i])/v[i]

        return(a)

In [4]:
CF(7,20)

array([2., 1., 1., 1., 4., 1., 1., 1., 4., 1., 1., 1., 4., 1., 1., 1., 4.,
       1., 1., 1.])

In [187]:
%%time
l = []
cf_length = 500
acf_length = 1000
acf_check_length = 10

for i in np.arange(1, 10001):    
    if np.sqrt(i) != np.floor(np.sqrt(i)):
        z = CF(i, cf_length)
        a = acf(z, acf_length)
        if not np.isnan(np.sum(a[:acf_check_length])):
            period = np.argwhere(a>0.95)[1,0]
        else:
            period=1
        l.append([i, period])

z_df = pd.concat([pd.DataFrame([e], columns=["n","period"]) for e in l])
z_df.reset_index(inplace=True, drop=True)

CPU times: user 47.3 s, sys: 307 ms, total: 47.6 s
Wall time: 48.5 s


In [188]:
#plt.plot(z_df["n"], z_df["period"])
#plt.show()
z_df.loc[z_df.period%2==1]

,n,period
0,2,1
2,5,1
6,10,1
9,13,5
12,17,1
...,...,...
9865,9965,37
9870,9970,49
9873,9973,17
9885,9985,61


In [193]:
z_df.loc[:20, "period"].values

array([1, 2, 1, 2, 4, 2, 1, 2, 2, 5, 4, 2, 1, 2, 6, 2, 6, 6, 4, 2, 1])

In [194]:
189+31

220